# Podnodo – ein hemdsärmeliger Workflow zur Podcastarchivierung auf Zenodo

Voraussetzung ist ein irgendwie abgeschlossenes Produkt (z.B. Staffel), da das Archivpaket im Nachhinein nicht mehr geändert werden kann.

Es muss zunächst ein DOI auf Zenodo registriert werden.

In [ ]:
import requests
from lxml import etree

In [ ]:
zenodo_doi =  # verpflichtende Angabe
mp3_folder = "folder" # optional, wenn mp3s in kein eigenes Verzeichnis sollen, einfach String leer lassen ("")
feed = # URL des Feeds

import os
if not os.path.isdir(mp3_folder):
    os.makedirs(mp3_folder)

In [ ]:
res = requests.get(feed)
tree = etree.fromstring(res.content)

for mp3 in tree.findall('.//enclosure'):
    print(mp3.attrib['url'])

    mp3_file = requests.get(mp3.attrib['url'])
    mp3_filename = mp3.attrib['url'].rpartition('/')[-1]
    
    if len(mp3_folder) > 0:
        with open(mp3_folder + "/" + mp3_filename, 'wb') as f:
            f.write(mp3_file.content)
        mp3.attrib['url'] = f"https://zenodo.org/records/{zenodo_doi}/files/{mp3_folder}/{mp3_filename}"
    else:
        with open(mp3_filename, 'wb') as f:
            f.write(mp3_file.content)
        mp3.attrib['url'] = f"https://zenodo.org/records/{zenodo_doi}/files/{mp3_filename}"
    
    #write modified feed
    tree = tree.getroottree()
    tree.write("feed.rss")